In [11]:
import pandas as pd

df = pd.read_csv("../../../databases/q1_database.csv")
df.sample(5)

,Avg Math Result,Avg Reading Result,Avg Science Result,BMMJ1,CNT_Group,CREATOOS,ESCS,FAMCON,HISEI,HOMEPOS,...,ST289Q06JA,ST289Q10WA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q05WA,ST290Q07WA,ST290Q09WA,ST291Q01JA,ST355Q02JA
23328,452.1319,429.2207,473.0841,NaN,Below Average,1.8024,0.1253,2.9168,NaN,0.3902,...,5.0,NaN,4.0,4.0,4.0,4.0,4.0,NaN,4.0,NaN
316455,563.6351,528.3555,595.5656,43.33,Above Average,0.6216,0.7831,1.6169,43.33,1.5886,...,5.0,NaN,NaN,1.0,1.0,NaN,NaN,4.0,NaN,NaN
310341,302.8094,319.3973,287.8741,17.00,Below Average,NaN,-1.1208,NaN,17.00,-1.7328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0
156907,312.2763,368.7793,349.2264,16.38,Below Average,NaN,-1.2275,NaN,46.38,-1.6961,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337257,665.9400,572.9639,675.1478,48.82,Above Average,-0.8105,-0.3166,NaN,48.82,0.3457,...,NaN,5.0,4.0,4.0,NaN,4.0,4.0,NaN,NaN,NaN


In [12]:
print("\n--- Missing Values ---")
missing = df.isnull().mean().sort_values(ascending=False)
print(missing[missing > 0.5])


--- Missing Values ---
ST289Q10WA    0.667423
ST289Q06JA    0.666929
ST289Q05WA    0.665966
ST289Q02JA    0.665162
ST355Q02JA    0.636909
ST291Q01JA    0.600930
ST290Q03WA    0.523012
ST290Q05WA    0.522340
ST290Q07WA    0.522268
ST290Q02WA    0.521932
ST290Q01WA    0.520752
ST290Q09WA    0.520304
dtype: float64
